# Workshop 1 - Question and Answers
In this workshop, you will learning how to write prompts and feed them into LLMs. You
will also be learning how to use different prompt techniques to improve the response
from the LLM.

## Loading and Exploring the Dataset
The workshop will be using [`facebook/ExploreToM`](https://huggingface.co/datasets/facebook/ExploreToM) dataset from [HuggingFace](https://huggingface.co).

In [ ]:
%pip install datasets transformers torch torchdata 

In [4]:
# TODO: Load the following libraries: datasets
from datasets import load_dataset

In [5]:
# Dataset name
dataset_name = "facebook/ExploreToM"

In [8]:
# TODO: load and explore the dataset
dataset = load_dataset(dataset_name)

# TODO: number of rows in the dataset
print(dataset.shape)

# TODO: Keys in the dataset
print(dataset.keys())

# TODO: Feature names
print(dataset['train'].features)

# TODO: Display a single row
idx = 100
print(dataset['train'][idx])

print('\n-----------------------\n')

# print row in key/value pair
for k, v in dataset['train'][idx].items():
   print(f'{k}: {v}')


{'train': (13309, 18)}
dict_keys(['train'])
{'story_structure': Value(dtype='string', id=None), 'infilled_story': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'expected_answer': Value(dtype='string', id=None), 'qprop=params': Value(dtype='string', id=None), 'qprop=nth_order': Value(dtype='int64', id=None), 'qprop=non_unique_mental_state': Value(dtype='bool', id=None), 'sprop=is_false_belief_story_1st': Value(dtype='bool', id=None), 'sprop=is_false_belief_story_1st_and_2nd': Value(dtype='bool', id=None), 'sprop=story_accuracy_1st_raw': Value(dtype='float64', id=None), 'sprop=story_accuracy_1st_infilled': Value(dtype='float64', id=None), 'sprop=global_idx': Value(dtype='int64', id=None), 'param=story_type': Value(dtype='string', id=None), 'param=num_stories_total': Value(dtype='int64', id=None), 'param=max_sentences': Value(dtype='int64', id=None), 'param=num_people': Value(dtype='int64', id=None), 'param=num_moves': Value(dtype='int64', id=None), 'param=nu

In [10]:
# TODO: import pipeline
from transformers import pipeline

## `pipeline`
[`pipeline`](https://huggingface.co/docs/transformers/en/main_classes/pipelines) is an easy to use API to perform inferencing. It provides a wrapper for task-specific pipelines and abstracts most of the complexity by allowing you to focus on the model and the task. 

You can use `pipeline` to perform summarisation, image classification, audio generation, etc. You can find an exhaustive list of `pipeline` task [here](https://huggingface.co/docs/transformers/en/main_classes/pipelines#transformers.pipeline.task).

In [ ]:
# TODO: Summarise the text with the pipeline's default model

# Create a pipeline
qna = pipeline('question-answering')


No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cpu


In [18]:
idx = 30
context = dataset['train'][idx]['story_structure']
context = dataset['train'][idx]['infilled_story']
question = dataset['train'][idx]['question']
expected_answer = dataset['train'][idx]['expected_answer']

context = "The asymmetry of costs between drone-based attacks and conventional countermeasures was also vividly exemplified when Houthi fighters fired drones at US warships off the coast of Yemen, said Dr Ng."
question = "Was the US warships at the coast of Yemen?"

# Ask the question and context
result = qna(question=question, context=context)

print(f'context: {context}')
print(f'question: {question}')
print(f'geneated answer: {result}')
print(f'expected answer: {expected_answer}')

context: The asymmetry of costs between drone-based attacks and conventional countermeasures was also vividly exemplified when Houthi fighters fired drones at US warships off the coast of Yemen, said Dr Ng.
question: Was the US warships at the coast of Yemen?
geneated answer: {'score': 0.08029749989509583, 'start': 162, 'end': 184, 'answer': 'off the coast of Yemen'}
expected answer: leather satchel


## Manual Inference - Question and Answer
In this section, we will look at what `pipeline` does under the hood to perform its inference. This will give us a better understanding of the major steps involved.

In [19]:
# TODO: load tokenizer
from transformers import AutoTokenizer

## DistilBERT base cased distilled SQuAD
DistilBERT is a small, fast, cheap and light Transformer model trained by distilling BERT base. More details [here](https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).

In [20]:
model_name = "distilbert/distilbert-base-cased-distilled-squad"

In [21]:
# TODO: Create a tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [23]:
# TODO: Encode text
text = 'hello, world'
text = 'quick brown fox jump over the lazy dog'

encoded = tokenizer(text, return_tensors='pt')

print(encoded)


{'input_ids': tensor([[  101,  3613,  3058, 17594,  5152,  1166,  1103, 16688,  3676,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [24]:
# TODO: Encoding multiple texts
texts = [
   'hello, world',
   'big black bug bleed black blodd',
   'quick brown fox jump over the lazy dog'
]

encoded = tokenizer(texts, padding=True, return_tensors='pt')

print(encoded)

{'input_ids': tensor([[  101, 19082,   117,  1362,   102,     0,     0,     0,     0,     0],
        [  101,  1992,  1602, 15430, 24752,  1602,   171,  2858, 13976,   102],
        [  101,  3613,  3058, 17594,  5152,  1166,  1103, 16688,  3676,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [31]:
# TODO: Decode text
text_enc = encoded['input_ids'][0]
print(text_enc)

text_dec = tokenizer.decode(text_enc)
# Remove all special tokens
text_dec = tokenizer.decode(text_enc, skip_special_tokens=True)

print(text_dec)

tensor([  101, 19082,   117,  1362,   102,     0,     0,     0,     0,     0])
hello, world


## Working with LLMs
Create and instance of the Large Language Model (LLM). We will then create a simple
prompt, tokenize the prompt and feed the tokenized prompt to the LLM. The response
from the LLM will be decoded to human friendly text.

In [28]:
# TODO: Load libraries
from transformers import AutoModelForQuestionAnswering
import torch


In [29]:
# TODO: Load question answer model
model = AutoModelForQuestionAnswering.from_pretrained(model_name)


In [30]:
# TODO: Encode context and question
idx = 40
context = dataset['train'][idx]['story_structure']
question = dataset['train'][idx]['question']
expected_answer = dataset['train'][idx]['expected_answer']


In [41]:
# TODO: Tokenize the inputs
context_question_enc = [ context, question ]

inputs_enc = tokenizer(context, question, padding=True, return_tensors='pt')

print(inputs_enc)

print(inputs_enc.keys())


{'input_ids': tensor([[  101, 23929,  8495,  1389,  2242,  1103,  1707,  1395,   119, 23929,
          8495,  1389,  1427,  1103,  5444,  1106,  1103, 22823,  2884,   117,
          1134,  1110,  1145,  1388,  1107,  1103,  1707,  1395,   119, 23929,
          8495,  1389,  1427,  1103,  5444,  1106,  1103,  5439,  2068, 20492,
           117,  1134,  1110,  1145,  1388,  1107,  1103,  1707,  1395,   119,
          1799,  1142,  2168,  1108,  5664,   117,  6010,  9491,  1142,  2168,
          1107,  3318,   113,  1105,  1178,  1142,  2168,   114,   119,  6010,
          2242,  1103,  1707,  1395,   119, 20587,  2242,  1103,  1707,  1395,
           119,  6010,  1427,  1103,  5444,  1106,  1103,  5828,  5092,  9055,
           117,  1134,  1110,  1145,  1388,  1107,  1103,  1707,  1395,   119,
           102,  1130,  1134, 12461,  1209, 23929,  8495,  1389,  3403,  1111,
          1103,  5444,   136,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [42]:
result = model(inputs_enc['input_ids'], inputs_enc['attention_mask'])
print(result.keys())

print(result)

odict_keys(['start_logits', 'end_logits'])
QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[ -7.2662,  -8.9783, -11.9269, -12.6365, -10.8424, -10.9652, -10.8457,
         -12.5836, -10.9463,  -6.8022, -11.2528, -11.9217,  -8.4287,  -8.7599,
          -8.2561, -10.4833,  -6.9664,  -5.5114,  -9.5050, -11.5037, -10.7829,
         -11.6356, -11.7965, -11.6061, -10.8338, -10.4030, -10.0082, -12.3106,
          -8.9441,  -4.8724, -10.9543, -11.3210,  -7.1960,  -7.7169,  -7.6047,
          -8.6834,  -3.2407,  -1.9408,  -5.2579,  -7.7145, -10.1194,  -9.8470,
         -10.7596, -11.0527, -10.1784,  -9.4579,  -9.3249,  -9.3893, -11.7014,
          -9.9611,  -9.2894, -10.3074, -10.7931, -11.8876, -11.7950, -10.9179,
          -5.2110, -10.5952, -10.4168, -11.1634, -10.5719, -10.0169, -11.2725,
         -11.7600, -11.5089, -11.0465, -11.3901, -11.3931,  -7.5745,  -4.0237,
          -9.9737,  -9.8611,  -9.5033, -11.2365,  -7.7369,  -3.7837,  -9.4820,
          -9.8372,  -9.5677, -11.47

In [44]:
ans_start = torch.argmax(result.start_logits)
ans_end = torch.argmax(result.end_logits) + 1

print(ans_start, ans_end)
answer_id = inputs_enc['input_ids'][0][ans_start: ans_end]
print(answer_id)

tensor(37) tensor(40)
tensor([ 5439,  2068, 20492])


In [45]:
answer = tokenizer.decode(answer_id)

print(answer)

leather satchel


In [35]:
# Ensure minimum and maximum token length in the answer
def ensure_size(input_ids, answer, min_length = 2, max_length = 5):
   ans_start = torch.argmax(answer['start_logits'])
   ans_end = torch.argmax(answer['end_logits']) + 1
   ans_length = ans_end - ans_start
   if ans_length < min_length:
      ans_end = min(ans_start + min_length, len(input_ids[0]))
   elif ans_length > max_length:
      ans_end = ans_start + max_length
   ans_start = max(0, ans_start)
   ans_end = min(len(input_ids[0]), ans_end)
   return (ans_start, ans_end)

In [ ]:
# TODO Return a minimum of 5 tokens


In [ ]:
# TODO: Try this your self

context = """
Dickens wrote A Christmas Carol during a period when the British were exploring and re-evaluating past Christmas traditions, 
including carols, and newer customs such as cards and Christmas trees. He was influenced by the experiences of his own youth and 
by the Christmas stories of other authors, including Washington Irving and Douglas Jerrold. Dickens had written three Christmas 
stories prior to the novella, and was inspired following a visit to the Field Lane Ragged School, one of several establishments for 
London's street children. The treatment of the poor and the ability of a selfish man to redeem himself by transforming into a more 
sympathetic character are the key themes of the story. There is discussion among academics as to whether this is a fully secular 
story or a Christian allegory.
"""

question = "How many stories has Dickens wrote?"

